In [ ]:
for ii in 1:6
    #addprocs(["heathred"])
end

for ii in 1:4
    #addprocs(["amon"], dir="")
    #addprocs(["zeus"], dir="")
    #addprocs(["jove"], dir="")
end

for ii in 1:10
    #addprocs(["uggp"], dir="")
end

addprocs(6)
workers()

In [ ]:
include("ClusterSoup.jl")
@everywhere using RecursiveAutoencoders

In [ ]:
using Pipe
function pz(x :: AbstractArray)
    println(typeof(x), ": ", size(x))
end
macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

In [ ]:
using PyCall
@pyimport nltk
function tokenize(sentence::String)
    convert(Array{String,1},nltk.word_tokenize(sentence))
end

In [ ]:
include("load_embeddings.jl")

In [ ]:
LL,word_indexes, indexed_words =  load_embeddings("embeddings-scaled.EMBEDDING_SIZE=200.txt");
size(LL) |> println
word_indexes |> typeof |> println
indexed_words |> typeof |> println

In [ ]:
using Iterators
@pyimport nltk.corpus as nltk_corpus
n_training = 4261
training_sents = @pipe nltk_corpus.brown[:sents]() |> filter(s->1<length(s)<=5, _) |> take(_,n_training)  |> collect |> convert(Vector{Vector{String}},_);


In [ ]:
training_sents |> size

In [ ]:
chunk_data(:training_sents, training_sents)

In [ ]:
@everywhere function unpack!(rae::RAE, θ::Vector)
    W_e_len = length(rae.W_e)
    b_e_len = length(rae.b_e)
    W_d_len = length(rae.W_d)
    b_d_len = length(rae.b_d)
    W_e_shape = size(rae.W_e)
    W_d_shape = size(rae.W_d)
    
    rae.W_e = reshape(θ[1: W_e_len],W_e_shape)
    rae.b_e = θ[W_e_len+1: W_e_len+b_e_len]
    rae.W_d = reshape(θ[W_e_len+b_e_len+1: W_e_len+b_e_len+W_d_len],W_d_shape)
    rae.b_d = θ[W_e_len+b_e_len+W_d_len+1: end]
    
    rae
end

@everywhere function pack(rae::RAE)
    [rae.W_e[:],rae.b_e, rae.W_d[:],rae.b_d[:]] 
end

@everywhere function pack(∇W_e::Matrix{Float64}, ∇b_e::Vector{Float64}, ∇W_d::Matrix{Float64}, ∇b_d::Vector{Float64})
    [∇W_e[:], ∇b_e, ∇W_d[:], ∇b_d] 
end

#--------------------------------------------------------

function loss!(θ::Vector)  
    error("loss not defined")
end

function loss_grad!(θ::Vector, storage::Vector) 
    error("loss_grad not defined")
end

rae_outer = RAE(LL,word_indexes,indexed_words);
set_global(:rae_outer,rae_outer)
function loss_and_loss_grad!(θ::Vector, grad::Vector)   
    grad[:] = 0
    for pid in workers()
        remotecall(pid, θv->unpack!(rae_outer, θv),θ) 
    end
    function loss_and_loss_grad(ss::Words)
        tree, act_tree, pp, err = eval_to_tree(rae_outer, ss)
        ∇s=BPTS(rae_outer,act_tree)
        [pack(∇s...), err]
    end
    
    ret = prechunked_mapreduce(:training_sents, loss_and_loss_grad, (+)) 
    grad[:] = ret[1:end-1]
    err=ret[end]
    
    grad[:]/=length(training_sents)
    err/=length(training_sents)
    err
end



In [ ]:
using NLopt

f_call_count = 0
function tracking_loss_and_loss_grad!(θ::Vector, grad::Vector)
    global f_call_count
    f_call_count+=1
    f_val = loss_and_loss_grad!(θ, grad)   
    println(f_call_count, '\t',f_val,'\t',norm(grad))
    f_val
end
#:LD_LBFGS, :LD_MMA
opt = Opt(:LD_MMA, length(pack(rae_outer)))
ftol_abs!(opt,1e-9)
maxtime!(opt, 6000)
min_objective!(opt, tracking_loss_and_loss_grad!)
(optf,optx,ret) = optimize!(opt,pack(rae_outer))


In [ ]:
optf

In [ ]:
#rae_outer = remotecall_fetch(4, dummy->rae_outer,nothing);


In [ ]:
using Optim #https://github.com/JuliaOpt/Optim.jl
f=DifferentiableFunction(loss!,loss_grad!,loss_and_loss_grad!)

In [ ]:
#Profile.clear()
#Profile.init(delay=0.1)
tic()
res = optimize(f, pack(rae_outer), method=:l_bfgs, show_trace = true, store_trace = true, iterations = 10);
time_take = toc()

In [ ]:
#==
    method::ASCIIString
    initial_x::Array{T,N}
    minimum::Array{T,N}
    f_minimum::Float64
    iterations::Int
    iteration_converged::Bool
    x_converged::Bool
    xtol::Float64
    f_converged::Bool
    ftol::Float64
    gr_converged::Bool
    grtol::Float64
    trace::OptimizationTrace
    f_calls::Int
    g_calls::Int
=#
@printval time_take
@printval res.f_calls 
@printval res.g_calls 
@printval res.x_converged 
@printval res.iterations
@printval res.f_minimum
@printval res.gr_converged
@printval res.trace



https://github.com/JuliaLang/julia/blob/master/doc/manual/profile.rst Actual instructions on profiling



In [ ]:
using ProfileView
ProfileView.view()

 ##3000: 1<len(sentence)<=5
 - **base:** elapsed time: 138.752550969 seconds (34239757952 bytes allocated, 83.92% gc time)
 - **base 2 (after restart)** elapsed time: 232.798843691 seconds (58940288804 bytes allocated, 83.25% gc time)
 
 
 - **2 sys after fix:**
  - **1st** elapsed time: 209.497508123 seconds (51858148 bytes allocated, 0.16% gc time)
  - **2nd** elapsed time: 207.234529931 seconds (20805144 bytes allocated, 0.08% gc time) 
  
 - **12 (on 3 Amon, 4 Heathred, 5 Motsugo**
  - **1st** elapsed time: 29.078831144 seconds (68477788 bytes allocated, 0.58% gc time)
  - **2nd** elapsed time: 27.259917889 seconds (63650168 bytes allocated, 3.03% gc time)
  
 - **16**
  - **1st**  elapsed time: 31.49018094 seconds (213737316 bytes allocated, 3.27% gc time)
  - **2nd** elapsed time: 23.265633784 seconds (104136464 bytes allocated, 2.41% gc time)
  
  ##23954: 1<len(sentence)<15
  - **16** elapsed time: 3034.431581365 seconds (113266660 bytes allocated, 0.04% gc time)
  - **24, for 15 iterations** elapsed time: 2478.921464409 seconds (701113364 bytes allocated, 0.18% gc time)
  

In [ ]:
function cosine_dist(a,b)
    (a⋅b)/(norm(a)*norm(b))
end

function neighbour_dists(cc::Vector{Float64}, globe::Matrix{Float64})
    [cosine_dist(cc, globe[:,ii]) for ii in 1:size(globe,2)]
end


function show_best(rae::RAE,ĉ::Embedding, nbest=20)
    candidates=neighbour_dists(ĉ,rae.L)   
    best_cands = [ (findfirst(candidates,score), score)
                    for score in select(candidates,1:nbest, rev=true)[1:nbest]]
    vcat([[rae.indexed_words[ii] round(score,2)] for (ii,score) in best_cands]...)
end

function show_bests(rae::RAE,ĉs::Embeddings, nbest=20)
    hcat([show_best(rae,ĉs[:,ii],nbest) for ii in 1:size(ĉs,2)]...)
end


In [ ]:
sent = "dangerous trends predict failure"
sent_toks = tokenize(sent)

tree, act_tree, pp, score_total = eval_to_tree(rae_outer,sent_toks);
println(tree)

ĉs = unfold(rae_outer,tree,pp)
bests= show_bests(rae_outer, ĉs)


In [ ]:
bests[1:4,1:2:end]